## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**3050. Pizza Toppings Cost Analysis (Medium)**

**Table: Toppings**

| Column Name  | Type    | 
|--------------|---------| 
| topping_name | varchar | 
| cost         | decimal |

topping_name is the primary key for this table.
Each row of this table contains topping name and the cost of the topping. 

**Write a solution to calculate the total cost of all possible 3-topping pizza combinations from a given list of toppings. The total cost of toppings must be rounded to 2 decimal places.**

**Note:**
- Do not include the pizzas where a topping is repeated. For example, ‘Pepperoni, Pepperoni, Onion Pizza’.
- Toppings must be listed in alphabetical order. For example, 'Chicken, Onions, Sausage'. 'Onion, Sausage, Chicken' is not acceptable.

Return the result table ordered by total cost in descending order and combination of toppings in ascending order.

The result format is in the following example.

**Example 1:**

**Input:**

**Toppings table:**

| topping_name | cost |
|--------------|------|
| Pepperoni    | 0.50 |
| Sausage      | 0.70 |
| Chicken      | 0.55 |
| Extra Cheese | 0.40 |

**Output:**
| pizza                          | total_cost | 
|--------------------------------|------------|
| Chicken,Pepperoni,Sausage      | 1.75       |  
| Chicken,Extra Cheese,Sausage   | 1.65       |
| Extra Cheese,Pepperoni,Sausage | 1.60       |
| Chicken,Extra Cheese,Pepperoni | 1.45       | 

**Explanation:**
There are only four different combinations possible with the three topings:
- Chicken, Pepperoni, Sausage: Total cost is $1.75 (Chicken $0.55, Pepperoni $0.50, Sausage $0.70).
- Chicken, Extra Cheese, Sausage: Total cost is $1.65 (Chicken $0.55, Extra Cheese $0.40, Sausage $0.70).
- Extra Cheese, Pepperoni, Sausage: Total cost is $1.60 (Extra Cheese $0.40, Pepperoni $0.50, Sausage $0.70).
- Chicken, Extra Cheese, Pepperoni: Total cost is $1.45 (Chicken $0.55, Extra Cheese $0.40, Pepperoni $0.50).

Output table is ordered by the total cost in descending order.

In [0]:
toppings_data_3050 = [
    ("Pepperoni", 0.50),
    ("Sausage", 0.70),
    ("Chicken", 0.55),
    ("Extra Cheese", 0.40),
]

toppings_columns_3050 = ["topping_name", "cost"]
toppings_df_3050 = spark.createDataFrame(toppings_data_3050, toppings_columns_3050)
toppings_df_3050.show()

+------------+----+
|topping_name|cost|
+------------+----+
|   Pepperoni| 0.5|
|     Sausage| 0.7|
|     Chicken|0.55|
|Extra Cheese| 0.4|
+------------+----+



In [0]:
t1_3050 = toppings_df_3050.alias("t1_3050")
t2_3050 = toppings_df_3050.alias("t2_3050")
t3_3050 = toppings_df_3050.alias("t3_3050")

In [0]:
t1_3050.join(t2_3050, col("t1_3050.topping_name") < col("t2_3050.topping_name")) \
        .join(t3_3050, col("t2_3050.topping_name") < col("t3_3050.topping_name")) \
        .select(
            concat_ws(",", 
                    t1_3050["topping_name"], 
                    t2_3050["topping_name"], 
                    t3_3050["topping_name"]
            ).alias("pizza"),
            round(
                t1_3050["cost"] + t2_3050["cost"] + t3_3050["cost"], 2
            ).alias("total_cost")
        ) \
        .orderBy(col("total_cost").desc(), col("pizza").asc()).display()


pizza,total_cost
"Chicken,Pepperoni,Sausage",1.75
"Chicken,Extra Cheese,Sausage",1.65
"Extra Cheese,Pepperoni,Sausage",1.6
"Chicken,Extra Cheese,Pepperoni",1.45
